## Imports

In [ ]:
from glob import glob
from openpyxl import load_workbook
from random import randint
from google.colab import files

## Configuration

In [ ]:
#mode = "lines"
#mode = "phrases"
mode = "sentences"

In [ ]:
if mode == "phrases":
  match_lines = False
  word_split_token = " "
  characters_to_replace_with_word_split_token = ",!?\n"

In [ ]:
if mode == "sentences":
  match_lines = False
  word_split_token = "."
  characters_to_replace_with_word_split_token = "\n"

In [ ]:
if mode == "lines":
  match_lines = True
  word_split_token = "."
  characters_to_replace_with_word_split_token = "\n"

In [ ]:
min_phrase_length = 8

# File Upload

In [ ]:
input_files = files.upload()

In [ ]:
input_files.keys()

## Book Chopping

In [ ]:
def num_words(token, a=2, b=7):
  if token == ".":
    return 1
  else:
    return randint(a, b) + randint(a, b)

In [ ]:
def clean(phrase: str):
  phrase = phrase.replace("â€", ",")
  for char in characters_to_replace_with_word_split_token:
    phrase = phrase.replace(char, word_split_token)
  return phrase.strip()

In [ ]:
def get_words(input_files:list, word_split_token:str):
  words = []
  for book in input_files.values():
    words.extend(clean(str(book)).split(word_split_token))
  return words

In [ ]:
def get_phrases_random(input_files, word_split_token):
  phrases = []
  words = get_words(input_files, word_split_token)
  while True:
    try:
      n = num_words(word_split_token)
      phrase = [words.pop(0) for _ in range(n)]
    except IndexError:
      phrase = words
      break
    finally:
      phrase = " ".join(phrase).strip()
      if len(phrase) > min_phrase_length:
        phrases.append(phrase)
  return phrases

In [ ]:
def get_phrases_lines(input_files):
  lines = []
  for book in input_files.values():
    book = str(book)
    for line in book:
      line = clean(line)
      if len(line) > min_phrase_length:
        lines.append(line)
  return lines

In [ ]:
def get_phrases(input_files, match_lines=False, word_split_token=" "):
  if match_lines:
    return get_phrases_lines(input_files)
  else:
    return get_phrases_random(input_files, word_split_token)

In [ ]:
phrases = get_phrases(input_files, match_lines, word_split_token)

In [ ]:
phrases[999]

## File Writing

wb = load_workbook(filename="template.xlsx")
phrases_sheet = wb["phrases"]
for i, phrase in enumerate(phrases):
    phrases_sheet.append((i, phrase))
wb.save(output_file_name)